In [31]:
import pandas as pd
import numpy as np

In [20]:
#columns = ['current_state', 'action', 'next_state', 'probability']
#df = pd.DataFrame(pd.read_csv('mdp_1d_statedata.xls', header = None),columns=columns)
df = pd.read_csv('mdp_1d_statedata.xls', header = None)
type(df)

pandas.core.frame.DataFrame

In [23]:
df.head()

,0,1,2,3
0,State 1,Action A,State 2,0.3
1,State 1,Action A,State 3,0.7
2,State 1,Action B,State 1,0.5
3,State 1,Action B,State 2,0.5
4,State 2,Action A,State 1,0.4


In [24]:
columns = ['current_state', 'action', 'next_state', 'probability']
df.columns = columns

In [25]:
df.head()

,current_state,action,next_state,probability
0,State 1,Action A,State 2,0.3
1,State 1,Action A,State 3,0.7
2,State 1,Action B,State 1,0.5
3,State 1,Action B,State 2,0.5
4,State 2,Action A,State 1,0.4


In [39]:
states = df['current_state'].unique()
print(states)
print(type(states))

['State 1' 'State 2' 'State 3']
<class 'numpy.ndarray'>


In [59]:
random_samples = []
num_samples = 100000  # Number of samples to generate

for _ in range(num_samples):
    random_state = np.random.choice(states)
    #print(random_state)
    sample_action = np.random.choice((df[df['current_state'] == random_state]).action.unique())
    #print(sample_action)
    sample_row = df[(df['current_state'] == random_state) & (df['action'] == sample_action)].sample(n=1, weights='probability').iloc[0]

    # Extract the values from the selected row
    current_state = sample_row['current_state']
    action = sample_row['action']
    next_state = sample_row['next_state']

    # Add the sample to the list
    random_samples.append([current_state, action, next_state])
    #print(random_samples)

In [60]:
df_samples = pd.DataFrame(random_samples, columns=['current_state', 'action', 'next_state'])
df_samples.head()

,current_state,action,next_state
0,State 3,Action B,State 3
1,State 2,Action B,State 2
2,State 2,Action A,State 3
3,State 2,Action A,State 3
4,State 1,Action A,State 3


In [61]:
df_empirical_probs = df_samples.groupby(['current_state', 'action', 'next_state']).size().reset_index(name='count')
print(df_empirical_probs)
df_empirical_count = df_samples.groupby(['current_state', 'action']).size().reset_index(name='count_total')
print(df_empirical_count)

df_empirical_probs = df_empirical_probs.merge(df_empirical_count, on=['current_state', 'action'])
df_empirical_probs['empirical_probability'] = df_empirical_probs['count'] / df_empirical_probs['count_total']
print(df_empirical_probs)


   current_state     action next_state  count
0        State 1   Action A    State 2   4910
1        State 1   Action A    State 3  11649
2        State 1   Action B    State 1   8381
3        State 1   Action B    State 2   8437
4        State 2   Action A    State 1   6718
5        State 2   Action A    State 3  10034
6        State 2   Action B    State 1   3305
7        State 2   Action B    State 2  13278
8        State 3   Action A    State 2  10081
9        State 3   Action A    State 3   6601
10       State 3   Action B    State 1   1680
11       State 3   Action B    State 3  14926
  current_state     action  count_total
0       State 1   Action A        16559
1       State 1   Action B        16818
2       State 2   Action A        16752
3       State 2   Action B        16583
4       State 3   Action A        16682
5       State 3   Action B        16606
   current_state     action next_state  count  count_total  \
0        State 1   Action A    State 2   4910        16559  

In [62]:
df_merged = df.merge(df_empirical_probs, on=['current_state', 'action', 'next_state'], how='left')
# Check if the empirical probabilities match the specified probabilities
df_merged['prob_diff'] = df_merged['probability'] - df_merged['empirical_probability']
all_probs_match = np.allclose(df_merged['probability'], df_merged['empirical_probability'])

# Display the comparison results
print(df_merged)
print("\nDo all probabilities match?", all_probs_match)

   current_state     action next_state  probability  count  count_total  \
0        State 1   Action A    State 2          0.3   4910        16559   
1        State 1   Action A    State 3          0.7  11649        16559   
2        State 1   Action B    State 1          0.5   8381        16818   
3        State 1   Action B    State 2          0.5   8437        16818   
4        State 2   Action A    State 1          0.4   6718        16752   
5        State 2   Action A    State 3          0.6  10034        16752   
6        State 2   Action B    State 1          0.2   3305        16583   
7        State 2   Action B    State 2          0.8  13278        16583   
8        State 3   Action A    State 2          0.6  10081        16682   
9        State 3   Action A    State 3          0.4   6601        16682   
10       State 3   Action B    State 1          0.1   1680        16606   
11       State 3   Action B    State 3          0.9  14926        16606   

    empirical_probabilit